## EE232E Project 2
### Problem 1

In [5]:
import os
for file in os.listdir("project_2_data"):
    print(file)

actor_movies.txt
actress_movies.txt
director_movies.txt
movie_genre.txt
movie_rating.txt
README.txt
